# Домашняя работа №2, сравнение теггеров

Работу выполнила Камская Милена, БКЛ-211

Обратим внимание на файл gold.conllu

В нем лежат 24 размеченных предложения, 235 слов. Я использовала **Universal POS tags**, чтобы в дальнейшем не усложнять себе жизнь, т.к. он часто используется и в других POS теггерах (более того, я возьму именно те теггеры, в которых используется upos).

Касательно сложных моментов:

1. __Заимствования__
        
      Я брала те слова, что редко присутсвуют в повседневной речи и с большой вероятностью не подчиняются стандартным правилам русского язык.
2. __Составные предлоги__

      Теггер может их разделить и начать неверно трактовать. К примеру, _за счет_ это полноценный ADP, а не ADP+NOUN
3. __Слова через дефис__

      Я брала те слова, которые нельзя разделять по дефису. Сюда входят и предлоги (_из-за_), и наречия (_по-человечески_), и существительные (_пол-лимона_), и глаголы с частицами (_Подойди-ка_), и междометия (_ей-богу_)
4. __Отглагольные предлоги__

      Я взяла отглагольные предлоги (к примеру, _благодаря_), т.к. их можно спутать с деепричастиями

### Stanza

In [ ]:
pip install stanza

In [ ]:
import stanza

nlp_stanza = stanza.Pipeline(lang='ru', processors='tokenize,pos')

In [119]:
def pos_by_stanza(file):
    with open('pos_by_stanza.txt', 'w', encoding='UTF-8') as output:
        with open(file, 'r', encoding='UTF-8') as input:
            f = input.readlines()
            for line in f:
                if 'text' in line:
                    sentences = nlp_stanza(line[9:-1])
                    for sent in sentences.sentences:
                        for token in sent.words:
                          output.write(token.text + '\t' + token.upos + '\n')

In [10]:
pos_by_stanza('gold.conllu')

### Natasha

In [ ]:
pip install natasha

In [120]:
from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)


segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [121]:
def pos_by_natasha(file):
    with open('pos_by_natasha.txt', 'w', encoding='UTF-8') as output:
        with open(file, 'r', encoding='UTF-8') as input:
            f = input.readlines()
            for line in f:
                if 'text' in line:
                    sent = Doc(line[9:-1])
                    sent.segment(segmenter)
                    sent.tag_morph(morph_tagger)
                    for token in sent.tokens:
                        output.write(token.text + '\t' + token.pos + '\n')

In [122]:
pos_by_natasha('gold.conllu')

### Spacy

In [ ]:
!python3 -m spacy download ru_core_news_sm

In [123]:
import spacy

In [124]:
nlp_spacy = spacy.load("ru_core_news_sm")

In [125]:
def pos_by_spacy(file):
    with open('pos_by_spacy.txt', 'w', encoding='UTF-8') as output:
        with open(file, 'r', encoding='UTF-8') as input:
            f = input.readlines()
            for line in f:
                if 'text' in line:
                    sentence = nlp_spacy(line[9:-1])
                    for token in sentence:
                        output.write(token.text + '\t' + token.pos_ + '\n')

In [126]:
pos_by_spacy('gold.conllu')

### Accuracy Test (точнее мои грустные попытки)

In [100]:
from sklearn.metrics import accuracy_score
from collections import defaultdict

def accuracy_testing(file_gold, file_pos):
    results = []
    golden = []
    d_gold = defaultdict(list)
    d_pred = defaultdict(list)

    with open(file_gold, 'r', encoding='UTF-8') as gold:
        for line in gold.readlines():
            if ('#' not in line) and (line[0] != '\n'):
                line = line.split('\t')
                d_gold[line[1]].append(line[3])
                golden.append(line[3])

    with open(file_pos, 'r', encoding='UTF-8') as pred:
        for pred_line in pred.readlines():
            pred_line = pred_line.strip('\n')
            pred_line = pred_line.split('\t')
            d_pred[pred_line[0]].append(pred_line[1])

    for el, pos in d_gold.items():
        if el in d_pred.keys():
            results.append(d_pred[el])
        else:
            results.append('error')

    print("Accuracy: %.4f" % accuracy_score(golden,results))

In [ ]:
res_stanza = accuracy_testing('gold.conllu', 'pos_by_stanza.txt')
res_spacy = accuracy_testing('gold.conllu', 'pos_by_spacy.txt')
res_natasha = accuracy_testing('gold.conllu', 'pos_by_natasha.txt')

Я понимаю, что проблема в том, что я сравниваю 2 разноразмерных списка, но починить это я не в силах. Если вдруг есть какие-то комментарии к тому, как я могу это сделать, то было бы очень славно послушать. Если таких комментариев нет, надо просто забить и ловить дзен

In [69]:
res_stanza

0.0

In [43]:
res_spacy

0.0

In [44]:
res_natasha

0.0

### Chunker

Я буду делать chunker для stanza

Я выделила 3 шаблона:
1. __"не"+VERB__

      Это сильно повлияет на результаты, т.к. рассматривать раздельно эти вещи нельзя. Именно частица _не_ играет огромную роль в изменении тональности (сравните: _понравился_ и _не понравился_)
2. __ADV+VERB__

      Аналогично прошлому пункту. (сравните: _хорошо смотрелся_ и _ужасно смотрелся_)
3. __VERB+ADV__

      Я, конечно, понимаю, что это очень похоже на прошлый шаблон, но, изучая отзывы, я нашла немалое количество примеров по типу _звучало ужасно_ и _звучало шикарно_, которые важны для более корректного определения тональности

Выполним все нужные импорты

In [102]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import pandas as pd
import csv
from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [103]:
def chunker(text):
    words_pos = []
    for sentence in sent_tokenize(text):
        stanza_app = nlp_stanza(sentence)
        for sent in stanza_app.sentences:
            for token in sent.words:
                words_pos.append([token.text, token.upos])

    bigrams = []
    for i in range(0, (len(words_pos)-1)):
        if words_pos[i][0] == 'не' and words_pos[i+1][1] == 'VERB':
            bigram = words_pos[i][0]+' '+words_pos[i+1][0]
            bigrams.append(bigram)

        if words_pos[i][1] == 'ADV' and words_pos[i+1][1] == 'VERB':
            bigram = words_pos[i][0]+' '+words_pos[i+1][0]
            bigrams.append(bigram)

        if words_pos[i][0] == 'VERB' and words_pos[i+1][1] == 'ADV':
            bigram = words_pos[i][0]+' '+words_pos[i+1][0]
            bigrams.append(bigram)

    return bigrams

Возьмем файлы из прошлой домашки, готовые к использованию

Теперь создадим 2 множества: в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных.

In [105]:
data = pd.read_csv('reviews_train.csv',sep=',', encoding='UTF-8')

text_positive = data[data['Sentiment'] == 'Positive']['Text']
text_negative = data[data['Sentiment'] == 'Negative']['Text']

In [106]:
def dict_for_set(text): #функция для словаря отдельная, потому что он нам потом понадобится (спойлер)
    all = []
    for line in text:
        w = line.split(' ')
        for word in w:
            all.append(word)

    d = {}
    for item in set(all):
        num = all.count(item)
        d[item] = num

    pure_d = {}
    for k, v in d.items():
        if v > 10: #здесь те слова, которые встречаются менее 11 раз я убираю, потому что они запутают модель
            pure_d[k] = v

    return pure_d

def set_of_words(text):
    d = dict_for_set(text)

    return set(d.keys())

In [107]:
s_pos = set_of_words(text_positive)
s_neg = set_of_words(text_negative)

Создадим списки с биграммами и словарик со словами для дальнейшего определения тональности

In [108]:
bigrams_pos = []
bigrams_neg = []

for line in text_positive:
    bigrams_detect = chunker(line)
    for elem in bigrams_detect:
        bigrams_pos.append(elem)

for line in text_negative:
    bigrams_detect = chunker(line)
    for elem in bigrams_detect:
        bigrams_neg.append(elem)

In [109]:
freq_lists = {}
freq_lists['Positive'] = dict_for_set(text_positive)
freq_lists['Negative'] = dict_for_set(text_negative)

Подготовим данные для тестирования выборки

In [110]:
data_test = open('reviews_test.csv', encoding='UTF-8')

test_dictionary = {}
for line in csv.reader(data_test):
    test_dictionary[line[0]] = line[1]

test_dictionary.pop('Text')

data_test.close

<function TextIOWrapper.close()>

Теперь создадим функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём и оценим его accuracy

In [111]:
def simple_sent_detect(freq_lists, text):
    counts = Counter()
    for sent, freq_list in freq_lists.items():
        freq_list = Counter(freq_list)
        for word in nltk.word_tokenize(text):
            counts[sent] += int(freq_list[word] > 0)
    return counts.most_common()

In [112]:
def accuracy_simple_sent_detect(freq_lists, dictionary):
    results = []  # сюда будем писать результаты
    gold = []     # сюда будем писать исходную тональность
    for line, sent in test_dictionary.items():
        predicted_lang = simple_sent_detect(freq_lists, line)[0][0]
        results.append(predicted_lang)
        gold.append(sent)
    print("RESULTS:")
    print("Accuracy: %.4f" % accuracy_score(results, gold))

In [113]:
accuracy_simple_sent_detect(freq_lists, test_dictionary)

RESULTS:
Accuracy: 0.6000


Теперь сделаем то же самое, только с биграммами

In [114]:
def simple_sent_detect_bigrams(bigrams_pos, bigrams_neg, text):

    positive = 0
    negative = 0

    for i in range(0, len(text.split())-1):
        big = text.split()[i]+' '+text.split()[i+1]
        if big in set(bigrams_pos):
            positive+=1
        elif big in set(bigrams_neg):
            negative+=1

    if text.split()[i] in s_pos:
        positive+=1
    elif text.split()[i] in s_neg:
        negative+=1

    if positive > negative:
        return 'Positive'
    elif positive < negative:
        return 'Negative'
    else:
        return 'Equal'

In [115]:
def accuracy_simple_sent_detect_bigrams(bigrams_pos, bigrams_neg,dictionary):
    results = []
    gold = []

    for text,grade in dictionary.items():
        pred = simple_sent_detect_bigrams(bigrams_pos, bigrams_neg, text)
        results.append(pred)
        gold.append(grade)

    print("RESULTS:")
    print("Accuracy: %.4f" % accuracy_score(results, gold))


In [116]:
accuracy_simple_sent_detect_bigrams(bigrams_pos, bigrams_neg,test_dictionary)

RESULTS:
Accuracy: 0.7000


Точность повысилась на одну десятую, однако у примененного метода есть минусы

In [117]:
print('Количество биграмм с положительной тональностью',len(set(bigrams_pos)),'\n','Количество биграмм с отрицательной тональностью',len(set(bigrams_neg)))

Количество биграмм с положительной тональностью 93 
 Количество биграмм с отрицательной тональностью 143


Как минимум, я не уравнивала количество элементов множеств биграмм с положительной и отрицательной тональностями.

Более того, выбранные шаблоны можно и нужно оспаривать
1. __"не"+VERB__

      _не могли нарадоваться_ (отметит как отрицательное, а на деле положительное)
2. __ADV+VERB__

      _хорошо смотрелся бы в борделе_ (отметит как положительное, а на деле отрицательное)
3. __VERB+ADV__

      _звучало шикарно для глухого_(отметит как положительное, а на деле отрицательное)

Нет ничего идеального, спасибо за внимание! До новых встреч.